In [78]:
import pandas as pd
import numpy as np
import glob
import re
from datetime import date, timedelta
import io
import requests

# Standard plotly imports
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [79]:
# # Load files from folder
# path = 'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports'
# all_files = glob.glob(path + "/*.csv")

# files = []

# for filename in all_files:
#     file = re.search(r'([0-9]{2}\-[0-9]{2}\-[0-9]{4})', filename)[0]
#     df = pd.read_csv(filename, index_col=None, header=0)
#     df['date'] = pd.to_datetime(file)
#     files.append(df)

# df = pd.concat(files, axis=0, ignore_index=True, sort=False)

In [88]:
# Load files from web
file_date = date(2020, 1, 22)
dates = []

while file_date <= date.today():
    dates.append(file_date)
    file_date += timedelta(days=1)
    
files = []
for file in dates:
    file = file.strftime("%m-%d-%Y")
    print(file)
    url = r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(file)
    raw_string = requests.get(url).content
    df = pd.read_csv(io.StringIO(raw_string.decode('utf-8')))
    df['date'] = pd.to_datetime(file)
    df.rename(columns={'Country_Region': 'Country/Region'}, inplace=True)
    if file == '03-22-2020':
        geo_us = df[(df['date'] == '2020-03-22') &
                    (df['Country/Region'] == 'US')].groupby('Province/State',
                                              as_index=False).agg({'Longitude': 'mean',
                                                                    'Latitude': 'mean'})
    if file > '03-22-2020' and b'404: Not Found\n' not in raw_string:
        df = df.merge(geo_us, left_on='Province_State', right_on='Province/State')
        df.drop(['Province_State', 'Lat', 'Long_'], axis=1, inplace=True)
    files.append(df)

df = pd.concat(files, axis=0, ignore_index=True, sort=False)

01-22-2020
01-23-2020
01-24-2020
01-25-2020
01-26-2020
01-27-2020
01-28-2020
01-29-2020
01-30-2020
01-31-2020
02-01-2020
02-02-2020
02-03-2020
02-04-2020
02-05-2020
02-06-2020
02-07-2020
02-08-2020
02-09-2020
02-10-2020
02-11-2020
02-12-2020
02-13-2020
02-14-2020
02-15-2020
02-16-2020
02-17-2020
02-18-2020
02-19-2020
02-20-2020
02-21-2020
02-22-2020
02-23-2020
02-24-2020
02-25-2020
02-26-2020
02-27-2020
02-28-2020
02-29-2020
03-01-2020
03-02-2020
03-03-2020
03-04-2020
03-05-2020
03-06-2020
03-07-2020
03-08-2020
03-09-2020
03-10-2020
03-11-2020
03-12-2020
03-13-2020
03-14-2020
03-15-2020
03-16-2020
03-17-2020
03-18-2020
03-19-2020
03-20-2020
03-21-2020
03-22-2020
03-23-2020
03-24-2020


In [89]:
df

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude,FIPS,Admin2,Last_Update,Active,Combined_Key,404: Not Found
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11099,Diamond Princess,US,NaN,49.0,0.0,0.0,2020-03-23,35.4437,139.6380,88888.0,NaN,2020-03-23 23:19:34,0.0,"Diamond Princess, US",NaN
11100,Grand Princess,Canada,NaN,13.0,0.0,0.0,2020-03-23,37.6489,-122.6655,NaN,NaN,2020-03-23 23:23:10,0.0,"Grand Princess, Canada",NaN
11101,Grand Princess,US,NaN,28.0,1.0,0.0,2020-03-23,37.6489,-122.6655,99999.0,NaN,2020-03-23 23:19:34,0.0,"Grand Princess, US",NaN
11102,Guam,US,NaN,29.0,1.0,0.0,2020-03-23,13.4443,144.7937,66000.0,NaN,2020-03-23 23:19:34,0.0,"Guam, US",NaN


In [90]:
# Rename countries with duplicate naming conventions
df['Country/Region'].replace('Mainland China', 'China', inplace=True)
df['Country/Region'].replace('Hong Kong SAR', 'Hong Kong', inplace=True)
df['Country/Region'].replace(' Azerbaijan', 'Azerbaijan', inplace=True)
df['Country/Region'].replace('Holy See', 'Vatican City', inplace=True)
df['Country/Region'].replace('Iran (Islamic Republic of)', 'Iran', inplace=True)
df['Country/Region'].replace('Taiwan*', 'Taiwan', inplace=True)
df['Country/Region'].replace('Korea, South', 'South Korea', inplace=True)
df['Country/Region'].replace('Viet Nam', 'Vietnam', inplace=True)
df['Country/Region'].replace('Macao SAR', 'Macau', inplace=True)
df['Country/Region'].replace('Russian Federation', 'Russia', inplace=True)
df['Country/Region'].replace('Republic of Moldova', 'Moldova', inplace=True)
df['Country/Region'].replace('Czechia', 'Czech Republic', inplace=True)
df['Country/Region'].replace('Congo (Kinshasa)', 'Congo', inplace=True)
df['Country/Region'].replace('Northern Ireland', 'United Kingdom', inplace=True)
df['Country/Region'].replace('Republic of Korea', 'North Korea', inplace=True)
df['Country/Region'].replace('Congo (Brazzaville)', 'Congo', inplace=True)
df['Country/Region'].replace('Taipei and environs', 'Taiwan', inplace=True)
df['Country/Region'].replace('Others', 'Cruise Ship', inplace=True)
df['Province/State'].replace('Cruise Ship', 'Diamond Princess cruise ship', inplace=True)
df['Province/State'].replace('From Diamond Princess', 'Diamond Princess cruise ship', inplace=True)

# Replace old reporting standards
df['Province/State'].replace('Chicago', 'Illinois', inplace=True)
df['Province/State'].replace('Chicago, IL', 'Illinois', inplace=True)
df['Province/State'].replace('Cook County, IL', 'Illinois', inplace=True)
df['Province/State'].replace('Boston, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace(' Norfolk County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Suffolk County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Middlesex County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Norwell County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Plymouth County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Norfolk County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Berkshire County, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Unknown Location, MA', 'Massachusetts', inplace=True)
df['Province/State'].replace('Los Angeles, CA', 'California', inplace=True)
df['Province/State'].replace('Orange, CA', 'California', inplace=True)
df['Province/State'].replace('Santa Clara, CA', 'California', inplace=True)
df['Province/State'].replace('San Benito, CA', 'California', inplace=True)
df['Province/State'].replace('Humboldt County, CA', 'California', inplace=True)
df['Province/State'].replace('Sacramento County, CA', 'California', inplace=True)
df['Province/State'].replace('Travis, CA (From Diamond Princess)', 'California', inplace=True)
df['Province/State'].replace('Placer County, CA', 'California', inplace=True)
df['Province/State'].replace('San Mateo, CA', 'California', inplace=True)
df['Province/State'].replace('Sonoma County, CA', 'California', inplace=True)
df['Province/State'].replace('Berkeley, CA', 'California', inplace=True)
df['Province/State'].replace('Orange County, CA', 'California', inplace=True)
df['Province/State'].replace('Contra Costa County, CA', 'California', inplace=True)
df['Province/State'].replace('San Francisco County, CA', 'California', inplace=True)
df['Province/State'].replace('Yolo County, CA', 'California', inplace=True)
df['Province/State'].replace('Santa Clara County, CA', 'California', inplace=True)
df['Province/State'].replace('San Diego County, CA', 'California', inplace=True)
df['Province/State'].replace('Travis, CA', 'California', inplace=True)
df['Province/State'].replace('Alameda County, CA', 'California', inplace=True)
df['Province/State'].replace('Madera County, CA', 'California', inplace=True)
df['Province/State'].replace('Santa Cruz County, CA', 'California', inplace=True)
df['Province/State'].replace('Fresno County, CA', 'California', inplace=True)
df['Province/State'].replace('Riverside County, CA', 'California', inplace=True)
df['Province/State'].replace('Shasta County, CA', 'California', inplace=True)
df['Province/State'].replace('Seattle, WA', 'Washington', inplace=True)
df['Province/State'].replace('Snohomish County, WA', 'Washington', inplace=True)
df['Province/State'].replace('King County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Unassigned Location, WA', 'Washington', inplace=True)
df['Province/State'].replace('Clark County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Jefferson County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Pierce County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Kittitas County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Grant County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Spokane County, WA', 'Washington', inplace=True)
df['Province/State'].replace('Tempe, AZ', 'Arizona', inplace=True)
df['Province/State'].replace('Maricopa County, AZ', 'Arizona', inplace=True)
df['Province/State'].replace('Pinal County, AZ', 'Arizona', inplace=True)
df['Province/State'].replace('Madison, WI', 'Wisconsin', inplace=True)
df['Province/State'].replace('San Antonio, TX', 'Texas', inplace=True)
df['Province/State'].replace('Lackland, TX', 'Texas', inplace=True)
df['Province/State'].replace('Lackland, TX (From Diamond Princess)', 'Texas', inplace=True)
df['Province/State'].replace('Harris County, TX', 'Texas', inplace=True)
df['Province/State'].replace('Fort Bend County, TX', 'Texas', inplace=True)
df['Province/State'].replace('Montgomery County, TX', 'Texas', inplace=True)
df['Province/State'].replace('Collin County, TX', 'Texas', inplace=True)
df['Province/State'].replace('Ashland, NE', 'Nebraska', inplace=True)
df['Province/State'].replace('Omaha, NE (From Diamond Princess)', 'Nebraska', inplace=True)
df['Province/State'].replace('Douglas County, NE', 'Nebraska', inplace=True)
df['Province/State'].replace('Portland, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Umatilla, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Klamath County, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Douglas County, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Marion County, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Jackson County, OR ', 'Oregon', inplace=True)
df['Province/State'].replace('Washington County, OR', 'Oregon', inplace=True)
df['Province/State'].replace('Providence, RI', 'Rhode Island', inplace=True)
df['Province/State'].replace('Providence County, RI', 'Rhode Island', inplace=True)
df['Province/State'].replace('Grafton County, NH', 'New Hampshire', inplace=True)
df['Province/State'].replace('Rockingham County, NH', 'New Hampshire', inplace=True)
df['Province/State'].replace('Hillsborough, FL', 'Florida', inplace=True)
df['Province/State'].replace('Sarasota, FL', 'Florida', inplace=True)
df['Province/State'].replace('Santa Rosa County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Broward County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Lee County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Volusia County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Manatee County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Okaloosa County, FL', 'Florida', inplace=True)
df['Province/State'].replace('Charlotte County, FL', 'Florida', inplace=True)
df['Province/State'].replace('New York City, NY', 'New York', inplace=True)
df['Province/State'].replace('Westchester County, NY', 'New York', inplace=True)
df['Province/State'].replace('Queens County, NY', 'New York', inplace=True)
df['Province/State'].replace('New York County, NY', 'New York', inplace=True)
df['Province/State'].replace('Nassau, NY', 'New York', inplace=True)
df['Province/State'].replace('Nassau County, NY', 'New York', inplace=True)
df['Province/State'].replace('Rockland County, NY', 'New York', inplace=True)
df['Province/State'].replace('Saratoga County, NY', 'New York', inplace=True)
df['Province/State'].replace('Suffolk County, NY', 'New York', inplace=True)
df['Province/State'].replace('Ulster County, NY', 'New York', inplace=True)
df['Province/State'].replace('Fulton County, GA', 'Georgia', inplace=True)
df['Province/State'].replace('Floyd County, GA', 'Georgia', inplace=True)
df['Province/State'].replace('Polk County, GA', 'Georgia', inplace=True)
df['Province/State'].replace('Cherokee County, GA', 'Georgia', inplace=True)
df['Province/State'].replace('Cobb County, GA', 'Georgia', inplace=True)
df['Province/State'].replace('Wake County, NC', 'North Carolina', inplace=True)
df['Province/State'].replace('Chatham County, NC', 'North Carolina', inplace=True)
df['Province/State'].replace('Bergen County, NJ', 'New Jersey', inplace=True)
df['Province/State'].replace('Hudson County, NJ', 'New Jersey', inplace=True)
df['Province/State'].replace('Clark County, NV', 'Nevada', inplace=True)
df['Province/State'].replace('Washoe County, NV', 'Nevada', inplace=True)
df['Province/State'].replace('Williamson County, TN', 'Tennessee', inplace=True)
df['Province/State'].replace('Davidson County, TN', 'Tennessee', inplace=True)
df['Province/State'].replace('Shelby County, TN', 'Tennessee', inplace=True)
df['Province/State'].replace('Montgomery County, MD', 'Maryland', inplace=True)
df['Province/State'].replace('Harford County, MD', 'Maryland', inplace=True)
df['Province/State'].replace('Denver County, CO', 'Colorado', inplace=True)
df['Province/State'].replace('Summit County, CO', 'Colorado', inplace=True)
df['Province/State'].replace('Douglas County, CO', 'Colorado', inplace=True)
df['Province/State'].replace('El Paso County, CO', 'Colorado', inplace=True)
df['Province/State'].replace('Delaware County, PA', 'Pennsylvania', inplace=True)
df['Province/State'].replace('Wayne County, PA', 'Pennsylvania', inplace=True)
df['Province/State'].replace('Montgomery County, PA', 'Pennsylvania', inplace=True)
df['Province/State'].replace('Fayette County, KY', 'Kentucky', inplace=True)
df['Province/State'].replace('Jefferson County, KY', 'Kentucky', inplace=True)
df['Province/State'].replace('Harrison County, KY', 'Kentucky', inplace=True)
df['Province/State'].replace('Marion County, IN', 'Indiana', inplace=True)
df['Province/State'].replace('Hendricks County, IN', 'Indiana', inplace=True)
df['Province/State'].replace('Ramsey County, MN', 'Minnesota', inplace=True)
df['Province/State'].replace('Carver County, MN', 'Minnesota', inplace=True)
df['Province/State'].replace('Fairfield County, CT', 'Connecticut', inplace=True)
df['Province/State'].replace('Charleston County, SC', 'South Carolina', inplace=True)
df['Province/State'].replace('Spartanburg County, SC', 'South Carolina', inplace=True)
df['Province/State'].replace('Kershaw County, SC', 'South Carolina', inplace=True)
df['Province/State'].replace('Davis County, UT', 'Utah', inplace=True)
df['Province/State'].replace('Honolulu County, HI', 'Hawaii', inplace=True)
df['Province/State'].replace('Tulsa County, OK', 'Oklahoma', inplace=True)
df['Province/State'].replace('Fairfax County, VA', 'Virginia', inplace=True)
df['Province/State'].replace('St. Louis County, MO', 'Missouri', inplace=True)
df['Province/State'].replace('Unassigned Location, VT', 'Vermont', inplace=True)
df['Province/State'].replace('Bennington County, VT', 'Vermont', inplace=True)
df['Province/State'].replace('Johnson County, IA', 'Iowa', inplace=True)
df['Province/State'].replace('Jefferson Parish, LA', 'Louisiana', inplace=True)
df['Province/State'].replace('Johnson County, KS', 'Kansas', inplace=True)
df['Province/State'].replace('Washington, D.C.', 'District of Columbia', inplace=True)

In [91]:
# # Interpolate values for missing South Korea data on March 11

# if len(df[(df['date'] == '03-10-2020') & (df['Country/Region'] == 'South Korea')]) == 0:
    
#     # interpolate missing values
#     confirmed = (df[(df['date'] == '03-09-2020') & (df['Country/Region'] == 'South Korea')]['Confirmed'].iloc[0] +
#                  df[(df['date'] == '03-11-2020') & (df['Country/Region'] == 'South Korea')]['Confirmed'].iloc[0]) / 2

#     deaths = (df[(df['date'] == '03-09-2020') & (df['Country/Region'] == 'South Korea')]['Deaths'].iloc[0] +
#               df[(df['date'] == '03-11-2020') & (df['Country/Region'] == 'South Korea')]['Deaths'].iloc[0]) / 2

#     recovered = (df[(df['date'] == '03-09-2020') & (df['Country/Region'] == 'South Korea')]['Recovered'].iloc[0] +
#                  df[(df['date'] == '03-11-2020') & (df['Country/Region'] == 'South Korea')]['Recovered'].iloc[0]) / 2

#     latitude = df[(df['date'] == '03-09-2020') & (df['Country/Region'] == 'South Korea')]['Latitude'].iloc[0]
#     longitude = df[(df['date'] == '03-09-2020') & (df['Country/Region'] == 'South Korea')]['Longitude'].iloc[0]
    
#     # insert missing values
#     df = df.append(df.loc[(df['date'] == '03-09-2020') &
#                           (df['Country/Region'] == 'South Korea')].assign(**{'date': pd.to_datetime('03-10-2020'),
#                                                                              'Confirmed': int(confirmed),
#                                                                              'Deaths': int(deaths),
#                                                                              'Recovered': int(recovered)}), ignore_index=True)

In [92]:
# South Korea data on March 10 seems to be mislabled as North Korea
df.loc[(df['Country/Region'] == 'North Korea') & (df['date'] == '03-10-2020'), 'Country/Region'] = 'South Korea'

In [93]:
df

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,date,Latitude,Longitude,FIPS,Admin2,Last_Update,Active,Combined_Key,404: Not Found
0,Anhui,China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,China,1/22/2020 17:00,14.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,China,1/22/2020 17:00,6.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,China,1/22/2020 17:00,1.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,China,1/22/2020 17:00,NaN,NaN,NaN,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11099,Diamond Princess,US,NaN,49.0,0.0,0.0,2020-03-23,35.4437,139.6380,88888.0,NaN,2020-03-23 23:19:34,0.0,"Diamond Princess, US",NaN
11100,Grand Princess,Canada,NaN,13.0,0.0,0.0,2020-03-23,37.6489,-122.6655,NaN,NaN,2020-03-23 23:23:10,0.0,"Grand Princess, Canada",NaN
11101,Grand Princess,US,NaN,28.0,1.0,0.0,2020-03-23,37.6489,-122.6655,99999.0,NaN,2020-03-23 23:19:34,0.0,"Grand Princess, US",NaN
11102,Guam,US,NaN,29.0,1.0,0.0,2020-03-23,13.4443,144.7937,66000.0,NaN,2020-03-23 23:19:34,0.0,"Guam, US",NaN


In [94]:
# Re-order the columns for readability
df = df[['date',
         'Country/Region',
         'Province/State',
         'Confirmed',
         'Deaths',
         'Recovered',
         'Latitude',
         'Longitude']]

# Fill missing values as 0; create Active cases column
df['Confirmed'] = df['Confirmed'].fillna(0).astype(int)
df['Deaths'] = df['Deaths'].fillna(0).astype(int)
df['Recovered'] = df['Recovered'].fillna(0).astype(int)
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [95]:
# Replace missing values for latitude and longitude
df['Latitude'] = df['Latitude'].fillna(df.groupby('Province/State')['Latitude'].transform('mean'))
df['Longitude'] = df['Longitude'].fillna(df.groupby('Province/State')['Longitude'].transform('mean'))

In [96]:
df

,date,Country/Region,Province/State,Confirmed,Deaths,Recovered,Latitude,Longitude,Active
0,2020-01-22,China,Anhui,1,0,0,31.825700,117.226400,1
1,2020-01-22,China,Beijing,14,0,0,40.182400,116.414200,14
2,2020-01-22,China,Chongqing,6,0,0,30.057200,107.874000,6
3,2020-01-22,China,Fujian,1,0,0,26.078900,117.987400,1
4,2020-01-22,China,Gansu,0,0,0,36.206833,103.602967,0
...,...,...,...,...,...,...,...,...,...
11099,2020-03-23,US,Diamond Princess,49,0,0,35.443700,139.638000,49
11100,2020-03-23,Canada,Grand Princess,13,0,0,37.648900,-122.665500,13
11101,2020-03-23,US,Grand Princess,28,1,0,37.648900,-122.665500,27
11102,2020-03-23,US,Guam,29,1,0,13.444300,144.793700,28


In [97]:
df['Country/Region'].unique()

array(['China', 'Hong Kong', 'Macau', 'Taiwan', 'US', 'Japan', 'Thailand',
       'South Korea', 'Singapore', 'Philippines', 'Malaysia', 'Vietnam',
       'Australia', 'Mexico', 'Brazil', 'Colombia', 'France', 'Nepal',
       'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast', 'Germany',
       'Finland', 'United Arab Emirates', 'India', 'Italy', 'UK',
       'Russia', 'Sweden', 'Spain', 'Belgium', 'Cruise Ship', 'Egypt',
       'Iran', 'Israel', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan',
       'Bahrain', 'Kuwait', 'Austria', 'Algeria', 'Croatia',
       'Switzerland', 'Pakistan', 'Georgia', 'Greece', 'North Macedonia',
       'Norway', 'Romania', 'Denmark', 'Estonia', 'Netherlands',
       'San Marino', 'Azerbaijan', 'Belarus', 'Iceland', 'Lithuania',
       'New Zealand', 'Nigeria', 'North Ireland', 'Ireland', 'Luxembourg',
       'Monaco', 'Qatar', 'Ecuador', 'Czech Republic', 'Armenia',
       'Dominican Republic', 'Indonesia', 'Portugal', 'Andorra', 'Latvia',
       'Morocco', 'Saud

In [98]:
df[df['Country/Region'] == 'US'].groupby(['date', 'Province/State'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum()

Confirmed  Deaths  Recovered  Active
date       Province/State                                      
2020-01-22 Washington              1       0          0       1
2020-01-23 Washington              1       0          0       1
2020-01-24 Illinois                1       0          0       1
           Washington              1       0          0       1
2020-01-25 Illinois                1       0          0       1
...                              ...     ...        ...     ...
2020-03-23 Virginia              254       6          0     248
           Washington           2221     109          0    2112
           West Virginia          16       0          0      16
           Wisconsin             425       5          0     420
           Wyoming                26       0          0      26

[1212 rows x 4 columns]

In [99]:
df[df['Country/Region'] == 'US'].groupby('date')[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum()

,Confirmed,Deaths,Recovered,Active
date,,,,
2020-01-22,1,0,0,1
2020-01-23,1,0,0,1
2020-01-24,2,0,0,2
2020-01-25,2,0,0,2
2020-01-26,5,0,0,5
...,...,...,...,...
2020-03-19,13680,200,108,13372
2020-03-20,19101,244,147,18710
2020-03-21,25493,307,171,25015


In [100]:
# active cases
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum() - \
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() - \
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Recovered'].sum()

43104

In [101]:
# confirmed
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()

43656

In [102]:
# recovered
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Recovered'].sum()

0

In [103]:
# deaths
df[(df['Country/Region'] == 'US') & (df['date'] == df['date'].iloc[-1])]['Deaths'].sum()

552

In [104]:
df[df['date'] == df['date'].iloc[-1]]['Confirmed'].sum()

44381

In [105]:
# fatality rate
'{:.2f}%'.format(100 *
                df[df['date'] == df['date'].iloc[-1]]['Deaths'].sum() /
                df[df['date'] == df['date'].iloc[-1]]['Confirmed'].sum())

'1.26%'

In [106]:
fig = go.Figure([go.Scatter(x=df[df['Country/Region'] == 'US'].groupby('date')['date'].first(),
                            y=df[df['Country/Region'] == 'US'].groupby('date')['Active'].sum())])
fig.show()

In [107]:
region = 'US'

print('Fatality rate: {:.2f}%'.format(100 *
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() /
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.update_layout(title="COVID-19 infections in {}".format(region),
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

Fatality rate: 1.26%


In [108]:
region = 'China'

print('Fatality rate: {:.2f}%'.format(100 *
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() /
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.update_layout(title="COVID-19 infections in {}".format(region),
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in longlong_scalars



Fatality rate: nan%


In [109]:
region = 'Italy'

print('Fatality rate: {:.2f}%'.format(100 *
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() /
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.update_layout(title="COVID-19 infections in {}".format(region),
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in longlong_scalars



Fatality rate: nan%


In [110]:
region = 'South Korea'

print('Fatality rate: {:.2f}%'.format(100 *
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() /
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.update_layout(title="COVID-19 infections in {}".format(region),
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in longlong_scalars



Fatality rate: nan%


In [111]:
region = 'Spain'

print('Fatality rate: {:.2f}%'.format(100 *
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Deaths'].sum() /
                                     df[(df['Country/Region'] == region) &
                                        (df['date'] == df['date'].iloc[-1])]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.update_layout(title="COVID-19 infections in {}".format(region),
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in longlong_scalars



Fatality rate: nan%


In [112]:
print('Fatality rate: {:.2f}%'.format(100 *
                                     df[df['date'] == df['date'].iloc[-1]]['Deaths'].sum() /
                                     df[df['date'] == df['date'].iloc[-1]]['Confirmed'].sum()))

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df.groupby('date')['date'].first(),
                y=df.groupby('date')['Confirmed'].sum(),
                name="Total Confirmed"))


fig.add_trace(go.Scatter(
                x=df.groupby('date')['date'].first(),
                y=df.groupby('date')['Active'].sum(),
                name="Active cases"))

fig.add_trace(go.Scatter(
                x=df.groupby('date')['date'].first(),
                y=df.groupby('date')['Recovered'].sum(),
                name="Recovered"))

fig.add_trace(go.Scatter(
                x=df.groupby('date')['date'].first(),
                y=df.groupby('date')['Deaths'].sum(),
                name="Deaths"))

fig.update_layout(title="COVID-19 infections Worldwide",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

Fatality rate: 1.26%


In [113]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'China'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'China'].groupby('date')['Active'].sum(),
                name="China",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'Italy'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'Italy'].groupby('date')['Active'].sum(),
                name="Italy",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'South Korea'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'South Korea'].groupby('date')['Active'].sum(),
                name="South Korea",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'US'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'US'].groupby('date')['Active'].sum(),
                name="US",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'Spain'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'Spain'].groupby('date')['Active'].sum(),
                name="Spain",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'France'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'France'].groupby('date')['Active'].sum(),
                name="France",
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df[df['Country/Region'] == 'Germany'].groupby('date')['date'].first(),
                y=df[df['Country/Region'] == 'Germany'].groupby('date')['Active'].sum(),
                name="Germany",
                opacity=0.8))

fig.update_layout(title="Active COVID-19 cases",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [114]:
fig = go.Figure()
countries = ['China', 'Italy', 'South Korea', 'US', 'Spain', 'France', 'Germany']

for country in countries:
    fig.add_trace(go.Scatter(
                    x=df[df['Country/Region'] == country].groupby('date')['date'].first(),
                    y=df[df['Country/Region'] == country].groupby('date')['Active'].sum(),
                    name=country,
                    opacity=0.8))

fig.update_layout(title="Active COVID-19 cases",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [115]:
fig = go.Figure()
for region in ['China', 'Italy', 'US', 'Spain', 'France', 'Germany', 'South Korea']:
    fig.add_trace(go.Scatter(
                    x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                    y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                    name=region,
                    hoverinfo='x+y+z+text+name',
                    stackgroup='one'))

fig.update_layout(title="COVID-19 Active Cases Worldwide",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [116]:
fig = go.Figure()
for region in df['Country/Region'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) & (df['Country/Region'] == region)]['Confirmed'].sum() > 500:
        fig.add_trace(go.Scatter(
                        x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                        y=df[df['Country/Region'] == region].groupby('date')['Active'].sum(),
                        name=region,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Active Cases Worldwide (Countries with greater than 100 confirmed cases)",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [117]:
fig = go.Figure()
for region in df['Country/Region'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) & (df['Country/Region'] == region)]['Confirmed'].sum() > 100:
        fig.add_trace(go.Scatter(
                        x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                        y=df[df['Country/Region'] == region].groupby('date')['Confirmed'].sum(),
                        name=region,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Confirmed Cases Worldwide (Countries with greater than 100 confirmed cases)",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [118]:
fig = go.Figure()
for region in df['Country/Region'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) & (df['Country/Region'] == region)]['Recovered'].sum() > 5:
        fig.add_trace(go.Scatter(
                        x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                        y=df[df['Country/Region'] == region].groupby('date')['Recovered'].sum(),
                        name=region,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Recovered Cases Worldwide (Countries with greater than 5 recovered cases)",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [119]:
fig = go.Figure()
for region in df['Country/Region'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) & (df['Country/Region'] == region)]['Deaths'].sum() > 5:
        fig.add_trace(go.Scatter(
                        x=df[df['Country/Region'] == region].groupby('date')['date'].first(),
                        y=df[df['Country/Region'] == region].groupby('date')['Deaths'].sum(),
                        name=region,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Deaths Worldwide (countries with greater than 5 deaths)",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [120]:
fig = go.Figure()
for state in df[df['Country/Region'] == 'US']['Province/State'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) &
          (df['Country/Region'] == 'US') &
          (df['Province/State'] == state)]['Confirmed'].sum() >= 1:
        fig.add_trace(go.Scatter(
                        x=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['date'].first(),
                        y=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['Active'].sum(),
                        name=state,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Active Cases in the United States",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [121]:
fig = go.Figure()
for state in df[df['Country/Region'] == 'US']['Province/State'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) &
          (df['Country/Region'] == 'US') &
          (df['Province/State'] == state)]['Confirmed'].sum() >= 1:
        fig.add_trace(go.Scatter(
                        x=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['date'].first(),
                        y=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['Confirmed'].sum(),
                        name=state,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Confirmed Cases in the United States",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [122]:
fig = go.Figure()
for state in df[df['Country/Region'] == 'US']['Province/State'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) &
          (df['Country/Region'] == 'US') &
          (df['Province/State'] == state)]['Deaths'].sum() >= 1:
        fig.add_trace(go.Scatter(
                        x=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['date'].first(),
                        y=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['Deaths'].sum(),
                        name=state,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Deaths in the United States",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [123]:
fig = go.Figure()
for state in df[df['Country/Region'] == 'US']['Province/State'].unique():
    if df[(df['date'] == df['date'].iloc[-1]) &
          (df['Country/Region'] == 'US') &
          (df['Province/State'] == state)]['Recovered'].sum() >= 1:
        fig.add_trace(go.Scatter(
                        x=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['date'].first(),
                        y=df[(df['Country/Region'] == 'US') &
                             (df['Province/State'] == state)].groupby('date')['Recovered'].sum(),
                        name=state,
                        hoverinfo='x+y+z+text+name',
                        stackgroup='one'))

fig.update_layout(title="COVID-19 Recovered Cases in the United States",
                  xaxis_title="Date",
                  yaxis_title="Number of Individuals")

fig.show()

In [124]:
data = df[df['date'] == df['date'].iloc[-1]].groupby('Country/Region').agg({'Confirmed': 'sum',
                                                                            'Longitude': 'mean',
                                                                            'Latitude': 'mean',
                                                                            'Country/Region': 'first'})
# Manually change some country centroids which are mislocated due to far off colonies
data.loc[data['Country/Region'] == 'US', 'Latitude'] = 39.810489
data.loc[data['Country/Region'] == 'US', 'Longitude'] = -98.555759

data.loc[data['Country/Region'] == 'France', 'Latitude'] = 46.2276
data.loc[data['Country/Region'] == 'France', 'Longitude'] = -3.4360

data.loc[data['Country/Region'] == 'United Kingdom', 'Latitude'] = 55.3781
data.loc[data['Country/Region'] == 'United Kingdom', 'Longitude'] = 2.2137

data.loc[data['Country/Region'] == 'Denmark', 'Latitude'] = 56.2639
data.loc[data['Country/Region'] == 'Denmark', 'Longitude'] = 9.5018

fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Country/Region'] + ': ' + data['Confirmed'].astype(str) + ' cases',
        mode = 'markers',
        marker_size = np.sqrt(data['Confirmed'] / 5),
#         marker_size = 100 * data['Confirmed'] / data['Confirmed'].max(),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Confirmed'],
                      cmax = data['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases by country',
                  geo=dict(scope='world',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showlakes=False,))
fig.show()

In [125]:
idx = -1
date = df['date'].unique()[idx]

data = df[df['date'] == date].groupby('Country/Region').agg({'Confirmed': 'sum',
                                                                            'Longitude': 'mean',
                                                                            'Latitude': 'mean',
                                                                            'Country/Region': 'first'})
if idx > 7:
    idx7 = idx - 7
else:
    idx7 = 0
    
data['share_of_last_week'] = ((df[df['date'] == date].groupby('Country/Region')['Confirmed'].sum() -
                               df[df['date'] == df['date'].unique()[idx7]].groupby('Country/Region')['Confirmed'].sum()) /
                               df[df['date'] == date].groupby('Country/Region')['Confirmed'].sum()) * 100
data['percentage'] = data['share_of_last_week'].fillna(0).apply(lambda x: '{:.0f}'.format(x))

# Manually change some country centroids which are mislocated due to far off colonies
data.loc[data['Country/Region'] == 'US', 'Latitude'] = 39.810489
data.loc[data['Country/Region'] == 'US', 'Longitude'] = -98.555759

data.loc[data['Country/Region'] == 'France', 'Latitude'] = 46.2276
data.loc[data['Country/Region'] == 'France', 'Longitude'] = -3.4360

data.loc[data['Country/Region'] == 'United Kingdom', 'Latitude'] = 55.3781
data.loc[data['Country/Region'] == 'United Kingdom', 'Longitude'] = 2.2137

data.loc[data['Country/Region'] == 'Denmark', 'Latitude'] = 56.2639
data.loc[data['Country/Region'] == 'Denmark', 'Longitude'] = 9.5018

fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Country/Region'] + ': ' + data['Confirmed'].astype(str) + ' total cases, ' + data['percentage'] + '% from previous week',
        mode = 'markers',
        marker_size = np.sqrt(data['Confirmed'] / 5),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['share_of_last_week'],
                      cmax = data['share_of_last_week'].max(),
                      colorbar_title="Share of recent cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases and share of new cases from the previous week',
                  geo=dict(scope='world',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showlakes=False,))
fig.show()

In [126]:
data = df[df['date'] == df['date'].iloc[-1]].groupby('Country/Region').agg({'Active': 'sum',
                                                                            'Longitude': 'mean',
                                                                            'Latitude': 'mean',
                                                                            'Country/Region': 'first'})
# Manually change some country centroids which are mislocated due to far off colonies
data.loc[data['Country/Region'] == 'US', 'Latitude'] = 39.810489
data.loc[data['Country/Region'] == 'US', 'Longitude'] = -98.555759

data.loc[data['Country/Region'] == 'France', 'Latitude'] = 46.2276
data.loc[data['Country/Region'] == 'France', 'Longitude'] = -3.4360

data.loc[data['Country/Region'] == 'United Kingdom', 'Latitude'] = 55.3781
data.loc[data['Country/Region'] == 'United Kingdom', 'Longitude'] = 3.4360

data.loc[data['Country/Region'] == 'Denmark', 'Latitude'] = 56.2639
data.loc[data['Country/Region'] == 'Denmark', 'Longitude'] = 9.5018

data.loc[data['Country/Region'] == 'Netherlands', 'Latitude'] = 52.1326
data.loc[data['Country/Region'] == 'Netherlands', 'Longitude'] = 5.2913

fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Country/Region'] + ': ' + data['Active'].astype(str) + ' cases',
        mode = 'markers',
        marker_size = np.sqrt(data['Active'] / 5),
#         marker_size = 100 * data['Confirmed'] / data['Confirmed'].max(),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Active'],
                      cmax = data['Active'].max(),
                      colorbar_title="Active Cases")))

fig.update_layout(title = 'Number of active cases by country',
                  geo=dict(scope='world',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showlakes=False,))
fig.show()

In [127]:
df2 = df.copy()

df2.loc[df2['Country/Region'] == 'Georgia', 'Province/State'] = 'Georgia (country)'
df2['Province/State'].fillna(df2['Country/Region'], inplace=True)

data = df2[df2['date'] == df2['date'].iloc[-1]].groupby('Province/State').agg({'Confirmed': 'sum',
                                                                               'Longitude': 'mean',
                                                                               'Latitude': 'mean',
                                                                               'Country/Region': 'first',
                                                                               'Province/State': 'first'})


fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Province/State'] + ', ' + data['Country/Region'] + ': ' + data['Confirmed'].astype(str),
        mode = 'markers',
        marker_size = np.sqrt(data['Confirmed'] / 5),
#         marker_size = 2 * np.log(data['Confirmed'] + 1),
#         marker_size = 100 * data['Confirmed'] / data['Confirmed'].max(),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Confirmed'],
                      cmax = data['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases by state or province (where available)',
                  geo=dict(scope='world',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=False,))
fig.show()

In [128]:
df2 = df.copy()

df2.loc[df2['Country/Region'] == 'Georgia', 'Province/State'] = 'Georgia (country)'
df2['Province/State'].fillna(df2['Country/Region'], inplace=True)

data = df2[df2['date'] == df2['date'].iloc[-1]].groupby('Province/State').agg({'Active': 'sum',
                                                                               'Longitude': 'mean',
                                                                               'Latitude': 'mean',
                                                                               'Country/Region': 'first',
                                                                               'Province/State': 'first'})


fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Province/State'] + ', ' + data['Country/Region'] + ': ' + data['Active'].astype(str),
        mode = 'markers',
        marker_size = np.sqrt(data['Active'] / 5),
#         marker_size = 2 * np.log(data['Confirmed'] + 1),
#         marker_size = 100 * data['Confirmed'] / data['Confirmed'].max(),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Active'],
                      cmax = data['Active'].max(),
                      colorbar_title="Active Cases")))

fig.update_layout(title = 'Number of active cases by state or province (where available)',
                  geo=dict(scope='world',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=False,))
fig.show()

In [129]:
df2 = df[df['Country/Region'] == 'US']

data = df2[df2['date'] == df2['date'].iloc[-1]].groupby('Province/State').agg({'Confirmed': 'sum',
                                                                               'Longitude': 'mean',
                                                                               'Latitude': 'mean',
                                                                               'Country/Region': 'first',
                                                                               'Province/State': 'first'})


fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Province/State'] + ', ' + data['Country/Region'] + ': ' + data['Confirmed'].astype(str),
        mode = 'markers',
        marker_size = (100 * data['Confirmed'] / data['Confirmed'].max()),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Confirmed'],
                      cmax = data['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases by state ',
                  geo=dict(scope='usa',
                           projection_type='albers usa',
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=True,))
fig.show()

In [130]:
df2 = df[df['Country/Region'] == 'US']

data = df2[df2['date'] == df2['date'].iloc[-1]].groupby('Province/State').agg({'Active': 'sum',
                                                                               'Longitude': 'mean',
                                                                               'Latitude': 'mean',
                                                                               'Country/Region': 'first',
                                                                               'Province/State': 'first'})


fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Province/State'] + ', ' + data['Country/Region'] + ': ' + data['Active'].astype(str),
        mode = 'markers',
        marker_size = (100 * data['Active'] / data['Active'].max()),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Active'],
                      cmax = data['Active'].max(),
                      colorbar_title="Active Cases")))

fig.update_layout(title = 'Number of active cases by state ',
                  geo=dict(scope='usa',
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=False,))
fig.show()

In [131]:
eu = ['Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia',
'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein',
'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway',
'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City']

In [132]:
df3 = df[df['Country/Region'].isin(eu)]

data = df3[df3['date'] == df3['date'].iloc[-1]].groupby('Country/Region').agg({'Active': 'sum',
                                                                               'Longitude': 'mean',
                                                                               'Latitude': 'mean',
                                                                               'Country/Region': 'first',
                                                                               'Province/State': 'first'})

data.loc[data['Country/Region'] == 'France', 'Latitude'] = 46.2276
data.loc[data['Country/Region'] == 'France', 'Longitude'] = -3.4360

data.loc[data['Country/Region'] == 'United Kingdom', 'Latitude'] = 55.3781
data.loc[data['Country/Region'] == 'United Kingdom', 'Longitude'] = 3.4360

data.loc[data['Country/Region'] == 'Denmark', 'Latitude'] = 56.2639
data.loc[data['Country/Region'] == 'Denmark', 'Longitude'] = 9.5018

data.loc[data['Country/Region'] == 'Netherlands', 'Latitude'] = 52.1326
data.loc[data['Country/Region'] == 'Netherlands', 'Longitude'] = 5.2913


fig = go.Figure(data=go.Scattergeo(
        lon = data['Longitude'],
        lat = data['Latitude'],
        text = data['Country/Region'] + ', ' + data['Country/Region'] + ': ' + data['Active'].astype(str),
        mode = 'markers',
        marker_size = (100 * data['Active'] / data['Active'].max()),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = data['Active'],
                      cmax = data['Active'].max(),
                      colorbar_title="Active Cases")))

fig.update_layout(title = 'Number of active cases by European country ',
                  geo=dict(scope='europe',
                           projection_type="natural earth",
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=False,))
fig.show()

In [133]:
temp = pd.read_csv('data/03-23-2020.csv')

df3 = temp[temp['Country_Region'] == 'US']


fig = go.Figure(data=go.Scattergeo(
        lon = df3['Long_'],
        lat = df3['Lat'],
        text = df3['Admin2'] + ', ' + df3['Country_Region'] + ': ' + df3['Confirmed'].astype(str),
        mode = 'markers',
        marker_size = (200 * df3['Confirmed'] / df3['Confirmed'].max()),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = df3['Confirmed'],
                      cmax = df3['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases by state ',
                  geo=dict(scope='usa',
                           projection_type='albers usa',
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=True,))
fig.show()

In [134]:
geo_us = df[(df['date'] == '2020-03-22') &
          (df['Country/Region'] == 'US')].groupby('Province/State',
                                                  as_index=False).agg({'Longitude': 'mean',
                                                                        'Latitude': 'mean'})

temp2 = pd.read_csv('data/03-23-2020.csv')

df4 = temp2[temp2['Country_Region'] == 'US'].groupby('Province_State', as_index=False).agg({'Confirmed': 'sum'})

df4 = df4.merge(geo_us, left_on='Province_State', right_on='Province/State')


fig = go.Figure(data=go.Scattergeo(
        lon = df4['Longitude'],
        lat = df4['Latitude'],
        text = df4['Province_State'] + ', ' + ': ' + df4['Confirmed'].astype(str),
        mode = 'markers',
        marker_size = (200 * df4['Confirmed'] / df4['Confirmed'].max()),
        marker = dict(reversescale = False,
                      autocolorscale = False,
                      symbol = 'circle',
                      line = dict(width=1, color='rgba(102, 102, 102)'),
                      colorscale = 'Reds',
                      cmin = 0,
                      color = df4['Confirmed'],
                      cmax = df4['Confirmed'].max(),
                      colorbar_title="Confirmed Cases")))

fig.update_layout(title = 'Number of cumulative confirmed cases by state ',
                  geo=dict(scope='usa',
                           projection_type='albers usa',
                           showland = True,
                           landcolor = "rgb(100, 125, 100)",
                           showocean = True,
                           oceancolor = "rgb(150, 150, 250)",
                           showcountries=True,
                           showsubunits=True,
                           showlakes=True,))
fig.show()

In [135]:
pd.to_datetime(df['date'].unique()[30]).strftime('%Y-%m-%d') == '2020-02-11'

False

In [136]:
pd.to_datetime(df['date'].unique()[60]).strftime('%Y-%m-%d')

'2020-03-22'

In [140]:
date = df['date'].unique()[61]
view = 'United States'

df_world_map = df[df['date'] == date].groupby('Country/Region').agg({'Confirmed': 'sum',
                                                                    'Longitude': 'mean',
                                                                    'Latitude': 'mean',
                                                                    'Country/Region': 'first'})

# if pd.to_datetime(date).strftime('%Y-%m-%d') > '2020-03-22' and view == 'United States':
#     geo_us = df[(df['date'] == '2020-03-22') &
#                 (df['Country/Region'] == 'US')].groupby('Province/State',
#                                               as_index=False).agg({'Longitude': 'mean',
#                                                                     'Latitude': 'mean'})
#     df_world_map = df_world_map[['Province/State']].merge(geo_us, left_on='Province_State', right_on='Province/State')

df_world_map

,Confirmed,Longitude,Latitude,Country/Region
Country/Region,,,,
Canada,13,8.486250,36.546300,Canada
Cruise Ship,712,139.638000,35.443700,Cruise Ship
US,43656,-91.921465,38.352584,US


In [139]:
geo_us.to_csv('geo_us.csv', index=False)

In [141]:
df_world_map[['Confirmed']].merge(geo_us, left_on='Country/Region', right_on='Province/State')

,Confirmed,Province/State,Longitude,Latitude
0,43656,US,-95.7129,37.0902
